Weak Lensing Null Test notebook.  

Chris W. and Keith B. for the SV bootcamp.  First version on 05/30/18.

In this notebook we use a the HSC 100 sq deg. Deep dataset to test:

- The agreement between the PSF model from PSFEX and the measured PSF from stars.
- Check shape correllation between stars and galaxies.

In [30]:
import glob
import numpy as np
import pandas as pd

from collections import OrderedDict

import lsst.daf.persistence as daf_persistence

import matplotlib.pyplot as pyplot
%matplotlib inline

Define a function to RA and DEC center of a patch in a tract.

In [22]:
def getPatchCenter(skymap, tract, patch):
    '''
    Find the center of the patch BBox in pixel coordinates and convert with tract WCS
    An example of the format is:
    tract = 1800
    patch = '4,4'
    '''
    
    # Convert strings to ints
    patch = list(map(int, patch.split(','))) 

    # Get WCS for this tract
    wcs = skymap.generateTract(tract).getWcs()

    # Get four corners of patch
    bbox = skymap.generateTract(tract).getPatchInfo(patch).getInnerBBox() 

    # Manually calculate the center of the patch and then return the values converted to degrees
    sky = wcs.pixelToSky(np.mean((bbox.getBeginX(), bbox.getEndX())), 
                         np.mean((bbox.getBeginY(), bbox.getEndY())))
    return sky[0].asDegrees(), sky[1].asDegrees()

First open the HSC datafiles with the Butler

In [24]:
depth = 'DEEP'
butler = daf_persistence.Butler('/datasets/hsc/repo/rerun/DM-13666/{:s}/'.format(depth))

# In the generation 2 butler it isn't possibe to introspect the metadata for any step after the initial
# data ingest.  This code parses the names of the files to build a dataframe for later inspection.

infiles = glob.glob('/datasets/hsc/repo/rerun/DM-13666/{:s}/deepCoadd-results/HSC-I/*/*/calexp-*.fits'
                    .format(depth))

# Get the skymap for the repo so we can understand where the tracks and patches are
skymap = butler.get('deepCoadd_skyMap')


# Now get the names of the tracts and patches based on the file parsing
tract_array = []
patch_array = []
for i in range(0, len(infiles)):
    parts = infiles[i].split('/')
    tract_array.append(int(parts[-3]))
    patch_array.append(parts[-2])

number_patches = len(patch_array)

# Create arrays to hold the RA and DEC of the centers of the patches
ra_center_array = np.empty(number_patches, dtype=np.float)
dec_center_array = np.empty(number_patches, dtype=np.float) 

# Loop through the patches and figure out where their centers are in degrees
for index, (tract, patch) in enumerate(zip(tract_array, patch_array)):
    ra_center_array[index], dec_center_array[index] = getPatchCenter(skymap, tract, patch)
    
print('Found {:d} patches'.format(number_patches))

Found 1303 patches


In [45]:
# Make a pandas dataframe with information on each tract and patch
data = OrderedDict({'tract': tract_array,
        'patch': patch_array, 
        'ra_center': ra_center_array, 
        'dec_center': dec_center_array})

df_tract_patch = pd.DataFrame(data=data)